In [18]:
import pandas as pd
import numpy as np
import httpx
from datetime import datetime

# Функции

## Функция для получения данных

In [8]:
async def fetch_bybit_data(url, params):
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, params=params, timeout=10.0)
            response.raise_for_status()
            result = response.json()
            return result["result"]

        except httpx.RequestError as e:
            return {"error": f"Request failed: {str(e)}"}
        except httpx.HTTPStatusError as e:
            return {"error": f"HTTP error: {str(e)}"}

## Функция для получения данных по свечам

In [14]:
async def get_kline(url,  symbol = "BTCUSDT", interval = "1", limit = 25):
    params = {
        "category": "spot",
        "symbol": symbol,
        "interval": interval,
        "limit": limit,
    }
    data = await fetch_bybit_data(url, params)

    return data

## Функция для приведения данных о свечах к единному формату

In [15]:
def format_kline_data(data):
    formatted_list = []
    list = data["list"]

    for elem in list:
        formatted_list.append(
            {
                "time": datetime.fromtimestamp(int(elem[0]) / 1000).strftime(
                    "%d-%m-%Y %H:%M:%S"
                ),
                "open": float(elem[1]),
                "high": float(elem[2]),
                "low": float(elem[3]),
                "close": float(elem[4]),
                "volume": float(elem[5]),
                "turnover": float(elem[6]),
            }
        )

    return formatted_list

## Функция для формирования датафрейма

In [24]:
def make_dataframe(data):
    dataframes = []

    for elem in data:
        elem["open"] = float(elem["open"])
        elem["close"] = float(elem["close"])
        elem["high"] = float(elem["high"])
        elem["low"] = float(elem["low"])
        elem["volume"] = float(elem["volume"])
        dataframes.append(pd.DataFrame(elem, index=[0]))

    initial_df = pd.concat(dataframes, ignore_index=True)

    return initial_df

# Bybit

## Получаем данные по свечам

In [25]:
url = "https://api.bybit.com/v5/market/kline"

kline_data = await get_kline(url, symbol="BTCUSDT", interval=1, limit=1000)
klines = format_kline_data(kline_data)

klines_df = make_dataframe(klines)

klines_df

,time,open,high,low,close,volume,turnover
0,19-12-2024 13:56:00,101708.41,101722.71,101694.11,101722.71,8.414443,8.558166e+05
1,19-12-2024 13:55:00,101713.96,101722.73,101691.75,101708.41,17.651862,1.795310e+06
2,19-12-2024 13:54:00,101694.44,101713.96,101673.43,101713.96,15.197696,1.545470e+06
3,19-12-2024 13:53:00,101739.19,101755.55,101694.43,101694.44,25.370773,2.580854e+06
4,19-12-2024 13:52:00,101790.29,101807.27,101689.55,101739.19,24.701046,2.513449e+06
...,...,...,...,...,...,...,...
995,18-12-2024 21:21:00,103605.44,103605.44,103399.99,103437.75,63.155588,6.537043e+06
996,18-12-2024 21:20:00,103568.70,103679.59,103560.11,103605.44,43.015561,4.456942e+06
997,18-12-2024 21:19:00,103563.75,103608.54,103466.97,103568.70,52.916857,5.479102e+06
998,18-12-2024 21:18:00,103663.61,103667.62,103529.88,103563.75,27.970512,2.897597e+06
